In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.python.client import device_lib
 

# Dataset Example

In [2]:
!nvidia-smi

Wed Feb 16 11:27:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A    0C    P8    N/A /  N/A |      0MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:GPU:0']


In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
BASE_DIR = os.path.abspath(os.path.join("cats_and_dogs", "training_set"))
CATS_DIR = os.path.join(BASE_DIR, "cats")
DOGS_DIR = os.path.join(BASE_DIR, "dogs")

BATCH_SIZE = 16
IMAGE_SIZE = (128, 128)

In [6]:
cat_images = [os.path.join(CATS_DIR, f) for f in os.listdir(CATS_DIR)]
dog_images = [os.path.join(DOGS_DIR, f) for f in os.listdir(DOGS_DIR)]

In [7]:
all_images = cat_images + dog_images

In [8]:
len(all_images)

8000

In [9]:
def preprocess_images(filename):
    file = tf.io.read_file(filename)
    
    img = tf.image.decode_jpeg(file)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = img / 255.0
    
    label = tf.strings.split(filename, sep = "\\")[-2] == "cats"
    label = tf.cast(label, tf.int32)
    
    return img, label

In [10]:
dataset = tf.data.Dataset.from_tensor_slices(all_images) \
    .shuffle(len(all_images)) \
    .map(preprocess_images) \
    .batch(BATCH_SIZE) \
    .prefetch(tf.data.AUTOTUNE) \
    .repeat()

In [11]:
for x in dataset:
    print(x)
    break

(<tf.Tensor: shape=(16, 128, 128, 3), dtype=float32, numpy=
array([[[[0.05490196, 0.04313726, 0.02352941],
         [0.05490196, 0.04313726, 0.02352941],
         [0.05098039, 0.03921569, 0.01960784],
         ...,
         [0.30584842, 0.22349548, 0.11761313],
         [0.26127452, 0.17892157, 0.07303922],
         [0.2546262 , 0.14874387, 0.06639093]],

        [[0.06233149, 0.05056679, 0.03095895],
         [0.06233149, 0.05056679, 0.03095895],
         [0.05840993, 0.04664522, 0.02703738],
         ...,
         [0.2725589 , 0.19020595, 0.0843236 ],
         [0.23463194, 0.152279  , 0.04639665],
         [0.27058825, 0.16470589, 0.08235294]],

        [[0.06666667, 0.05490196, 0.03529412],
         [0.06666667, 0.05490196, 0.03529412],
         [0.05882353, 0.04705882, 0.02745098],
         ...,
         [0.229519  , 0.14324449, 0.04912684],
         [0.23688282, 0.15060832, 0.05649067],
         [0.2484375 , 0.14255515, 0.06020221]],

        ...,

        [[0.1665945 , 0.17051607

In [12]:
model = Sequential([
    Input((128, 128, 3)),
    Conv2D(filters = 64, kernel_size = (5, 5), padding = "same", activation = "relu"),
    Conv2D(filters = 64, kernel_size = (5, 5), padding = "same", activation = "relu"),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(filters = 32, kernel_size = (3, 3), padding = "same", activation = "relu"),
    Conv2D(filters = 32, kernel_size = (3, 3), padding = "same", activation = "relu"),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(filters = 16, kernel_size = (3, 3), padding = "same", activation = "relu"),
    Conv2D(filters = 16, kernel_size = (3, 3), padding = "same", activation = "relu"),
    MaxPool2D(),
    Flatten(),
    Dense(64, activation = "relu"),
    Dropout(0.2),
    Dense(32, activation = "relu"),
    Dropout(0.1),
    Dense(1, activation = "sigmoid")
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      4864      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      102464    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        18464     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0

In [14]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [15]:
model.fit(dataset, steps_per_epoch = 500, epochs = 4)

Epoch 1/4
500/500 [==============================] - 87s 144ms/step - loss: 0.6783 - accuracy: 0.5815
Epoch 2/4
500/500 [==============================] - 76s 152ms/step - loss: 0.6190 - accuracy: 0.6658
Epoch 3/4
500/500 [==============================] - 78s 157ms/step - loss: 0.5571 - accuracy: 0.7164
Epoch 4/4
500/500 [==============================] - 80s 159ms/step - loss: 0.5063 - accuracy: 0.7527
